In [48]:
import pandas as pd
import os

movies1 = pd.read_csv("IMDB-Movie-Data.csv")
movies2 = pd.read_csv("tmdb-movies.csv")
movies3 = pd.read_csv("movies.csv")

Movies = [movies1,movies2,movies3]
result = pd.concat(Movies)
#movies.head()
useful = ["Title", "Genre", "Description", "Rating"]
##useful = ["original_title", "genres", "overview", "vote_average"]
df = result[useful]
df.head()

C:\Users\zodyac\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  if __name__ == '__main__':


,Title,Genre,Description,Rating
0,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,8.1
1,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",7.0
2,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,7.3
3,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",7.2
4,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,6.2


In [49]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
import time

t1 = time.time()

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
#metadata['overview'] = metadata['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df["Description"].values.astype('U'))

t2 = time.time()
print(t2 - t1)
#Output the shape of tfidf_matrix
tfidf_matrix.shape

0.8719151020050049


(16660, 34978)

In [50]:
tfidf_matrix

<16660x34978 sparse matrix of type '<class 'numpy.float64'>'
	with 426216 stored elements in Compressed Sparse Row format>

In [51]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [52]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(df.index, index=df['Title']).drop_duplicates()
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df['Title'].iloc[movie_indices]

In [54]:
indices[:20]

Title
Guardians of the Galaxy                     0
Prometheus                                  1
Split                                       2
Sing                                        3
Suicide Squad                               4
The Great Wall                              5
La La Land                                  6
Mindhorn                                    7
The Lost City of Z                          8
Passengers                                  9
Fantastic Beasts and Where to Find Them    10
Hidden Figures                             11
Rogue One                                  12
Moana                                      13
Colossal                                   14
The Secret Life of Pets                    15
Hacksaw Ridge                              16
Jason Bourne                               17
Lion                                       18
Arrival                                    19
dtype: int64

In [55]:
get_recommendations(input("Enter a movie : "))

Enter a movie : Hacksaw Ridge


8446                American Pop
1087                  Fort Bliss
11                Hidden Figures
8504          Courage Under Fire
1036          Courage Under Fire
10495    The Karate Kid, Part II
2386              One Man's Hero
816                         I.T.
939                Up in the Air
2569              American Movie
Name: Title, dtype: object